In [ ]:
import os 
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.info("Application started successfully.")

In [ ]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "etl"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("process_bronze_adventureWorks") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \

# set MinIO config
conf.set("fs.s3a.access.key", AWS_ACCESS_KEY) 
conf.set("fs.s3a.secret.key", AWS_SECRET_KEY) 
conf.set("fs.s3a.endpoint", AWS_S3_ENDPOINT) 
conf.set("fs.s3a.connection.ssl.enabled", "false") 
conf.set("fs.s3a.path.style.access", "true") 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

# Process bronze

In [ ]:
# create bronze namespace in nessie catalog
namespace = "BRONZE"
df = spark.sql("SHOW NAMESPACES in nessie")
df.show()
namespace_exists = df.filter(df.namespace == namespace).count() > 0

logger.info(f"namespace {namespace} exists: {namespace_exists}")
if not namespace_exists:
    spark.sql(f"CREATE NAMESPACE nessie.{namespace}").show()


In [ ]:
spark.sql(f"LIST REFERENCES IN nessie").show()
spark.sql(f"DROP BRANCH IF EXISTS {REF} IN nessie").show()
spark.sql(f"CREATE BRANCH {REF} IN nessie FROM main").show()
spark.sql(f"LIST REFERENCES IN nessie").show()

In [ ]:
# define list of data
source_list = [
    {
        "path": "s3a://seed/adventureWorks/currency_rate.csv",
        "iceberg_table": "bronze.currency_rate",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/currency.csv",
        "iceberg_table": "bronze.currency",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/customer.csv",
        "iceberg_table": "bronze.customer",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/product.csv",
        "iceberg_table": "bronze.product",
        "enabled": True
    },
    {
        "path": "s3a://seed/adventureWorks/sales.csv",
        "iceberg_table": "bronze.sales",
        "enabled": True
    }
]

In [ ]:
spark.sql(f"USE REFERENCE {REF} IN nessie").show()

for item in [item for item in source_list if item["enabled"]]:
    # Read CSV from MinIO
    logger.info(f'read csv from bucket: {item["path"]}')
    df = spark.read.option("header", "true") \
        .csv(item["path"])
    
    num_columns, num_rows = len(df.columns), df.count()

    # Write to Nessie table using createOrReplace
    table_name = item["iceberg_table"].upper()
    logger.info(f'writing data ({num_rows} rows, {num_columns} columns) into: {table_name}')
    try:
        df.writeTo(table_name).createOrReplace() 
        logger.info(f'Writing complete: {table_name}')
    except Exception as e:
        logger.error(f"Error writing into: {table_name}\n{str(e)}")


In [ ]:
spark.sql(f"MERGE BRANCH {REF} INTO main IN nessie").show()

In [ ]:
spark.sql(f"LIST REFERENCES IN nessie").show()

In [ ]:
spark.sql(f"DROP BRANCH IF EXISTS {REF} IN nessie").show()
spark.sql(f"USE REFERENCE main IN nessie").show()